# 1. Install YOLO package (seg models included in ultralytics-style API)

In [ ]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.5 MB/s eta 0:00:00


# 2. Imports

In [ ]:
import os, glob, time, json, math, gc, platform, inspect, zipfile
import torch
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# 3. Environment / device info

In [ ]:
print("PyTorch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("WARNING: Training on CPU will be extremely slow.")

PyTorch: 2.8.0+cu126 | CUDA: True
GPU: NVIDIA A100-SXM4-80GB


# 4. Load the base model we want to fine-tune
'yolo11-seg-m' here means:
- version 11 architecture
- task = segmentation
- size = medium

In [ ]:
model = YOLO("yolo11l-seg.pt")

In [ ]:
from google.colab import files
import os, zipfile, glob

YOLO_ROOT = "/content/data_yolo"
os.makedirs(YOLO_ROOT, exist_ok=True)

print("📦 Choose your dataset .zip…")
uploaded = files.upload()
assert uploaded, "No file uploaded."
zip_name = next(iter(uploaded))
zip_path = f"/content/{zip_name}"

print("🔓 Unzipping…")
with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall(YOLO_ROOT)

In [ ]:
def find_yolo_root(root):
    hits = []
    for dirpath, dirnames, filenames in os.walk(root):
        # check that we have train/valid[/test] with images+labels subfolders
        if all(os.path.isdir(os.path.join(dirpath, s)) for s in ("train","valid")):
            ok = True
            for s in ("train","valid"):
                img_dir = os.path.join(dirpath, s, "images")
                lab_dir = os.path.join(dirpath, s, "labels")
                if not (os.path.isdir(img_dir) and os.path.isdir(lab_dir)):
                    ok = False
                    break
            if ok:
                hits.append(dirpath)
    return sorted(hits, key=len)

cands = find_yolo_root(YOLO_ROOT)
assert cands, "Could not find YOLO-style dataset with train/valid/images+labels"
DATASET_DIR = cands[0]
print("✅ YOLO dataset root:", DATASET_DIR)

# Quick stats
for split in ("train","valid","test"):
    img_dir = os.path.join(DATASET_DIR, split, "images")
    lab_dir = os.path.join(DATASET_DIR, split, "labels")
    if os.path.isdir(img_dir):
        imgs = glob.glob(os.path.join(img_dir, "*"))
        labs = glob.glob(os.path.join(lab_dir, "*.txt"))
        print(f"{split}: {len(imgs)} images | {len(labs)} labels")

✅ YOLO dataset root: /content/data_yolo
train: 14965 images | 14965 labels
valid: 213 images | 213 labels
test: 213 images | 213 labels


In [ ]:
# ===============================
# YOLOv11-seg Hyperparameter Setup
# ===============================
import math, torch, os

# Target effective batch (total)
EFF_BATCH = 16

def suggest_per_gpu_batch():
    if not torch.cuda.is_available():
        return 2
    props = torch.cuda.get_device_properties(0)
    vram_gb = props.total_memory / (1024 ** 3)
    if vram_gb >= 30:      # A100
        return 16          # YOLO is lighter, can go higher
    elif vram_gb >= 20:
        return 8
    elif vram_gb >= 12:
        return 4
    else:
        return 2

PER_GPU_BATCH = suggest_per_gpu_batch()
GRAD_ACCUM = math.ceil(EFF_BATCH / PER_GPU_BATCH)

EPOCHS = 100
IMG_SIZE = 640
NUM_WORKERS = min(os.cpu_count(), 4)
PROJECT = "/content/drive/MyDrive/yolo11l_runs_WOCIMB"
os.makedirs(PROJECT, exist_ok=True)

print({
    "epochs": EPOCHS,
    "per_gpu_batch": PER_GPU_BATCH,
    "grad_accum": GRAD_ACCUM,
    "effective_batch": PER_GPU_BATCH * GRAD_ACCUM,
    "img_size": IMG_SIZE,
    "num_workers": NUM_WORKERS,
    "project": PROJECT
})

{'epochs': 100, 'per_gpu_batch': 16, 'grad_accum': 1, 'effective_batch': 16, 'img_size': 640, 'num_workers': 4, 'project': '/content/drive/MyDrive/yolo11l_runs_WOCIMB'}


# Start training

In [ ]:
results = model.train(
    data="/content/data_yolo/data.yaml",  # your converted dataset
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=PER_GPU_BATCH,
    project=PROJECT,
    name="yolo11_seg_l",
    # fl_gamma=1.5,     # focal loss
        cls=1.5,
    workers=NUM_WORKERS,
)

In [ ]:
# Evaluate the model on the validation set
metrics = model.val()

# Save the evaluation metrics to a file
metrics_path = os.path.join(PROJECT, "yolo11_seg_l", "metrics.json")
with open(metrics_path, "w") as f:
    json.dump(metrics.results_dict, f)

print(f"Evaluation metrics saved to: {metrics_path}")

# The best model weight is saved automatically during training in the runs directory
# The path to the best model is typically in PROJECT/name/weights/best.pt
best_model_path = os.path.join(PROJECT, "yolo11_seg_l", "weights", "best.pt")
print(f"Best model weight saved to: {best_model_path}")

##Detailed Evaluation metrics report code, can be run after the model weights are saved. This template can be used for all the models

In [ ]:
# ==========================
# Evaluate ONE YOLOv11-seg model + save metrics
# ==========================
!pip -q install ultralytics>=8.3.0

import os, csv, json, time
from pathlib import Path
from ultralytics import YOLO

# ---------- CONFIG (edit) ----------
WEIGHTS   = "/content/runs/segment/yolo11m-seg/weights/best.pt"   # model to evaluate
DATA_YAML = "/content/data_yolo/data.yaml"                        # your dataset yaml
SPLIT     = "test"                                                # "val" or "test"
IMGSZ     = 640
DEVICE    = 0                                                     # GPU id or "cpu"
# -----------------------------------

EVALS_DIR = Path("/content/evals"); EVALS_DIR.mkdir(parents=True, exist_ok=True)
CSV_PATH  = EVALS_DIR / "yolo11-lseg_metrics.csv"                   # cumulative log

run_name  = Path(WEIGHTS).parent.parent.name + "_" + Path(WEIGHTS).stem  # e.g., yolo11m-seg_best
model     = YOLO(WEIGHTS)

print(f"Evaluating {WEIGHTS} on {SPLIT}…")
res = model.val(
    data=DATA_YAML,
    split=SPLIT,
    imgsz=IMGSZ,
    device=DEVICE,
    workers=8,
    save_json=True,     # COCO predictions.json
    plots=True,         # confusion matrix, PR, etc. saved under runs/…
    project=PROJECT,
    name=f"eval_{run_name}",
)

# ---- Safely pull out metrics (seg + box) ----
def get(obj, attr, default=float("nan")):
    try:
        val = getattr(obj, attr)
        return float(val) if isinstance(val, (int, float)) else val
    except Exception:
        return default

seg = getattr(res, "seg", None)
box = getattr(res, "box", None)

metrics_row = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "run_name": run_name,
    "weights": str(WEIGHTS),
    "split": SPLIT,
    "imgsz": IMGSZ,

    # Segmentation metrics
    "seg_map_50_95": get(seg, "map"),
    "seg_map_50":    get(seg, "map50"),
    "seg_mp":        get(seg, "mp"),
    "seg_mr":        get(seg, "mr"),

    # Detection (boxes) metrics (sometimes also computed in seg tasks)
    "box_map_50_95": get(box, "map"),
    "box_map_50":    get(box, "map50"),
    "box_mp":        get(box, "mp"),
    "box_mr":        get(box, "mr"),

    # Speed (ms per image, averaged)
    "speed_preprocess_ms": getattr(res, "speed", {}).get("preprocess", float("nan")),
    "speed_inference_ms":  getattr(res, "speed", {}).get("inference", float("nan")),
    "speed_postprocess_ms":getattr(res, "speed", {}).get("postprocess", float("nan")),
}

# ---- Save CSV (append) ----
header = list(metrics_row.keys())
write_header = not CSV_PATH.exists()
with open(CSV_PATH, "a", newline="") as f:
    w = csv.DictWriter(f, fieldnames=header)
    if write_header: w.writeheader()
    w.writerow(metrics_row)

print(f"\nSaved summary → {CSV_PATH}")

# ---- Save detailed per-class AP as JSON (if available) ----
# Ultralytics exposes per-class AP array at .maps; we map it to class names from data.yaml
import yaml
with open(DATA_YAML, "r") as f:
    names = yaml.safe_load(f)["names"]

detail = {
    "run_name": run_name,
    "weights": str(WEIGHTS),
    "split": SPLIT,
    "seg": {
        "map_50_95": metrics_row["seg_map_50_95"],
        "map_50":    metrics_row["seg_map_50"],
        "per_class_ap_50_95": (
            dict(zip(names, getattr(seg, "maps", []))) if getattr(seg, "maps", None) is not None else {}
        ),
    },
    "box": {
        "map_50_95": metrics_row["box_map_50_95"],
        "map_50":    metrics_row["box_map_50"],
        "per_class_ap_50_95": (
            dict(zip(names, getattr(box, "maps", []))) if getattr(box, "maps", None) is not None else {}
        ),
    },
    "speed_ms": {
        "preprocess": metrics_row["speed_preprocess_ms"],
        "inference":  metrics_row["speed_inference_ms"],
        "postprocess":metrics_row["speed_postprocess_ms"],
    },
}

json_path = EVALS_DIR / f"{run_name}_details.json"
with open(json_path, "w") as f:
    json.dump(detail, f, indent=2)
print(f"Saved details → {json_path}")

print("\nDone. Plots & COCO JSON are under:")
print(Path("runs/segment") / f"eval_{run_name}")


In [ ]:
import zipfile
import os
from google.colab import files

folder_to_zip = "/content/drive/MyDrive/yolo11l_runs_WOCIMB"
output_zip = "/content/drive/MyDrive/yolo11l_runs_WOCIMB.zip"

with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
    for root, dirs, files_in_dir in os.walk(folder_to_zip):
        for file in files_in_dir:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, folder_to_zip)
            zf.write(file_path, arcname)

print(f"Zipped {folder_to_zip} to {output_zip}")

files.download(output_zip)

Zipped /content/drive/MyDrive/yolo11l_runs_WOCIMB to /content/drive/MyDrive/yolo11l_runs_WOCIMB.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>